<a href="https://colab.research.google.com/github/nikhil-nikss/DeepLearning/blob/main/DistilBERT.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

Sentiment Analysis using pretrained DistilBERT 

In [20]:
# installing transformers

!pip install transformers

In [21]:
# Importing important libraries

import os
import pandas as pd
import numpy as np
import torch
import transformers as ppb
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import cross_val_score,train_test_split

In [22]:
# checking current working directory

os.getcwd()

'/content'

In [23]:
# Reading dataset

data_file = pd.read_csv('train.tsv',delimiter="\t",header=None)

In [24]:
# Checking info of dataset

data_file.head(7)

,0,1
0,"a stirring , funny and finally transporting re...",1
1,apparently reassembled from the cutting room f...,0
2,they presume their audience wo n't sit still f...,0
3,this is a visually stunning rumination on love...,1
4,jonathan parker 's bartleby should have been t...,1
5,campanella gets the tone just right funny in t...,1
6,a fan film that for the uninitiated plays bett...,0


In [25]:
# importing pretrained distilBERT model and tokenization

model_class,tokenizer_class,pretrained_weights = (ppb.DistilBertModel,ppb.DistilBertTokenizer,'distilbert-base-uncased')

In [26]:
# Load pretrained model/tokenizer

tokenizer = tokenizer_class.from_pretrained(pretrained_weights)
model = model_class.from_pretrained(pretrained_weights)

In [27]:
# tokenization

tokenized = data_file[0].apply((lambda x:tokenizer.encode(x,add_special_tokens=True)))

In [28]:
# converted into list of ids

tokenized

0      [101, 1037, 18385, 1010, 6057, 1998, 2633, 182...
1      [101, 4593, 2128, 27241, 23931, 2013, 1996, 62...
2      [101, 2027, 3653, 23545, 2037, 4378, 24185, 10...
3      [101, 2023, 2003, 1037, 17453, 14726, 19379, 1...
4      [101, 5655, 6262, 1005, 1055, 12075, 2571, 376...
                             ...                        
995    [101, 2612, 1997, 5599, 1996, 11680, 2272, 200...
996    [101, 2023, 10722, 19068, 2080, 2323, 2031, 20...
997    [101, 2412, 2156, 2028, 1997, 2216, 22092, 200...
998    [101, 1996, 2143, 2003, 8052, 2005, 1996, 1592...
999    [101, 2005, 2087, 1997, 2049, 8333, 1010, 1996...
Name: 0, Length: 1000, dtype: object

In [29]:
# padding steps

max_len = 0
for i in tokenized.values:
  if len(i) > max_len:
    max_len = len(i)

padded = np.array([i + [0]*(max_len-len(i)) for i in tokenized.values])

In [30]:
# checking shape after padding

np.array(padded).shape

(1000, 59)

In [31]:
# Masking steps

attention_mask = np.where(padded != 0, 1, 0)
attention_mask.shape

(1000, 59)

In [32]:
# creating input tensors that would be fed to DistilBERT

input_ids = torch.tensor(np.array(padded))

with torch.no_grad():
  last_hidden_state = model(input_ids)

In [33]:
# Slice the output for the first position for all the sequences, take all hidden unit outputs

features = last_hidden_state[0][:,0,:].numpy()

In [34]:
features

array([[-0.28686032, -0.45407736,  0.1406745 , ...,  0.07227554,
         0.4919425 ,  0.10975704],
       [-0.14349563, -0.3003151 ,  0.285327  , ...,  0.01443081,
         0.4175469 ,  0.12710963],
       [-0.28867123, -0.07278142,  0.05309609, ...,  0.0021668 ,
         0.6752431 ,  0.18898168],
       ...,
       [-0.055615  , -0.289662  ,  0.26132864, ...,  0.07876023,
         0.54663616,  0.03366374],
       [-0.19624877, -0.32285482,  0.32309785, ...,  0.14977378,
         0.61592984,  0.0772965 ],
       [-0.18522048, -0.3353521 ,  0.15269779, ..., -0.06018153,
         0.5184222 ,  0.12131936]], dtype=float32)

In [37]:
# now training our logistic regression

labels = data_file[1]
X_train,X_test,y_train,y_test = train_test_split(features,labels)

In [39]:
# Logistic regression

classification_model = LogisticRegression()
classification_model.fit(X_train,y_train)

/usr/local/lib/python3.7/dist-packages/sklearn/linear_model/_logistic.py:940: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  extra_warning_msg=_LOGISTIC_SOLVER_CONVERGENCE_MSG)


LogisticRegression(C=1.0, class_weight=None, dual=False, fit_intercept=True,
                   intercept_scaling=1, l1_ratio=None, max_iter=100,
                   multi_class='auto', n_jobs=None, penalty='l2',
                   random_state=None, solver='lbfgs', tol=0.0001, verbose=0,
                   warm_start=False)

In [40]:
# checking test accuracy

classification_model.score(X_test,y_test)

0.82